## UNDERSTANDING DEEP LEARNING USING CNN

### ABSTRACT

**Context:**

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

The original MNIST dataset contains a lot of handwritten digits. Members of the AI/ML/Data Science community love this dataset and use it as a benchmark to validate their algorithms. In fact, MNIST is often the first dataset researchers try. "If it doesn't work on MNIST, it won't work at all", they said. "Well, if it does work on MNIST, it may still fail on others."

Zalando seeks to replace the original MNIST dataset

**Content:**

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.

To locate a pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27. The pixel is located on row i and column j of a 28 x 28 matrix.
For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below. 

**Labels:**

Each training and test example is assigned to one of the following labels:

0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot 

**Overview:**

Each row is a separate image
Column 1 is the class label.
Remaining columns are pixel numbers (784 total).
Each value is the darkness of the pixel (1 to 255)

Kaggle link to dataset - https://www.kaggle.com/zalando-research/fashionmnist

In [0]:
#Importing necessary libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

Using TensorFlow backend.


In [0]:
#Metadata
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 0

NO_EPOCHS = 2
BATCH_SIZE = 128

In [0]:
#Reading data from file
train_data = pd.read_csv('fashion-mnist_train.csv')
test_data = pd.read_csv('fashion-mnist_test.csv')

In [0]:
print("Fashion MNIST train -  rows:",train_data.shape[0]," columns:", train_data.shape[1])
print("Fashion MNIST test -  rows:",test_data.shape[0]," columns:", test_data.shape[1])

Fashion MNIST train -  rows: 60000  columns: 785
Fashion MNIST test -  rows: 10000  columns: 785


In [0]:
labels = {0 : "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat",
          5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle Boot"}

def get_classes_distribution(data):
    # Get the count for each label
    label_counts = data["label"].value_counts()

    # Get total number of samples
    total_samples = len(data)


    # Count the number of items in each class
    for i in range(len(label_counts)):
        label = labels[label_counts.index[i]]
        count = label_counts.values[i]
        percent = (count / total_samples) * 100
        print("{:<20s}:   {} or {}%".format(label, count, percent))

get_classes_distribution(train_data)

Ankle Boot          :   6000 or 10.0%
Bag                 :   6000 or 10.0%
Sneaker             :   6000 or 10.0%
Shirt               :   6000 or 10.0%
Sandal              :   6000 or 10.0%
Coat                :   6000 or 10.0%
Dress               :   6000 or 10.0%
Pullover            :   6000 or 10.0%
Trouser             :   6000 or 10.0%
T-shirt/top         :   6000 or 10.0%


In [0]:
def data_preprocessing(raw):
    out_y = keras.utils.to_categorical(raw.label, NUM_CLASSES)
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, IMG_ROWS, IMG_COLS, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

In [0]:
X, y = data_preprocessing(train_data)
X_test, y_test = data_preprocessing(test_data)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [0]:
print("Fashion MNIST train -  rows:",X_train.shape[0]," columns:", X_train.shape[1:4])
print("Fashion MNIST valid -  rows:",X_val.shape[0]," columns:", X_val.shape[1:4])
print("Fashion MNIST test -  rows:",X_test.shape[0]," columns:", X_test.shape[1:4])

Fashion MNIST train -  rows: 48000  columns: (28, 28, 1)
Fashion MNIST valid -  rows: 12000  columns: (28, 28, 1)
Fashion MNIST test -  rows: 10000  columns: (28, 28, 1)


In [0]:
#Creating an object of Sequential class
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               147584    
__________

In [0]:
#Fitting the model
train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/2
48000/48000 [==============================] - 56s 1ms/sample - loss: 0.5330 - acc: 0.8052 - val_loss: 0.4270 - val_acc: 0.8445
Epoch 2/2
48000/48000 [==============================] - 56s 1ms/sample - loss: 0.3450 - acc: 0.8741 - val_loss: 0.3472 - val_acc: 0.8735


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.337430632686615
Test accuracy: 0.8795


## Changing activation function to sigmoid

In [0]:
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='sigmoid',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='sigmoid'))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 54s 1ms/sample - loss: 1.2211 - acc: 0.5518 - val_loss: 0.7218 - val_acc: 0.7237
Epoch 2/2
48000/48000 [==============================] - 54s 1ms/sample - loss: 0.6381 - acc: 0.7575 - val_loss: 0.5894 - val_acc: 0.7628


In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5890627470970153
Test accuracy: 0.7659


Here, we see that changing activation function to sigmoid decreases the accuracy to 76% as compared to the activation function relu and softmax which gave an accuracy of about 87% Hence, we make use of relu and softmax as activation function and change the loss function to cosine proximity to see its effect.

## Changing loss function to cosine proximity 

In [0]:
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.8402 - acc: 0.8119 - val_loss: -0.8797 - val_acc: 0.8559
Epoch 2/2
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.8956 - acc: 0.8766 - val_loss: -0.8958 - val_acc: 0.8765


In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.901581658744812
Test accuracy: 0.8841


Here, changing loss function to cosine proxity leads to the test accuracy of 88% that is slightly greater than the cross entropy loss function which was 87% . Now, let us use both cosine proximity and cross entropy as our loss function and change the number of epochs to see its effect on the accuracy.

## Changing number of Epochs to 5

In [0]:
NO_EPOCHS = 5
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.8339 - acc: 0.8030 - val_loss: -0.8783 - val_acc: 0.8560
Epoch 2/5
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.8939 - acc: 0.8740 - val_loss: -0.8979 - val_acc: 0.8777
Epoch 3/5
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.9082 - acc: 0.8918 - val_loss: -0.9047 - val_acc: 0.8878
Epoch 4/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9173 - acc: 0.9027 - val_loss: -0.9162 - val_acc: 0.9007
Epoch 5/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9245 - acc: 0.9110 - val_loss: -0.9174 - val_acc: 0.9031


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.9201883563041687
Test accuracy: 0.9064


For loss function - cosine proximity, we see that test accuracy increased to 90% with loss of about -0.92. Now, let us the its effect on loss function - Cross Entropy

In [0]:
NO_EPOCHS = 5
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 56s 1ms/sample - loss: 0.5272 - acc: 0.8101 - val_loss: 0.3718 - val_acc: 0.8683
Epoch 2/5
48000/48000 [==============================] - 55s 1ms/sample - loss: 0.3357 - acc: 0.8788 - val_loss: 0.3408 - val_acc: 0.8760
Epoch 3/5
48000/48000 [==============================] - 55s 1ms/sample - loss: 0.2889 - acc: 0.8956 - val_loss: 0.2848 - val_acc: 0.8963
Epoch 4/5
48000/48000 [==============================] - 54s 1ms/sample - loss: 0.2556 - acc: 0.9062 - val_loss: 0.2713 - val_acc: 0.9020
Epoch 5/5
48000/48000 [==============================] - 54s 1ms/sample - loss: 0.2275 - acc: 0.9167 - val_loss: 0.2665 - val_acc: 0.9038


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.25092555755376816
Test accuracy: 0.9076


Here, we see that the accuracy does not increase much while the test loss increases to 0.25. Hence, we continue with loss function - cosine proximity. Now, let us increase the number of epochs and see if it has any effect.

## Changing number of Epochs to 7

In [0]:
NO_EPOCHS = 7
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/7
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.8378 - acc: 0.8076 - val_loss: -0.8831 - val_acc: 0.8618
Epoch 2/7
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.8922 - acc: 0.8720 - val_loss: -0.9000 - val_acc: 0.8806
Epoch 3/7
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9066 - acc: 0.8902 - val_loss: -0.9067 - val_acc: 0.8903
Epoch 4/7
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9173 - acc: 0.9023 - val_loss: -0.9130 - val_acc: 0.8961
Epoch 5/7
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.9243 - acc: 0.9115 - val_loss: -0.9164 - val_acc: 0.9024
Epoch 6/7
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9304 - acc: 0.9189 - val_loss: -0.9120 - val_acc: 0.8937
Epoch 7/7
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9363 - acc: 0.9261 - val_lo

In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.9222356794357299
Test accuracy: 0.9074


Increasing the epochs to 7, increases the test accuracy to 0.9074 which was earlier 0.9064 for 5 epochs. The increase in the accuracy is not much and we see that it plateaus, so it is better to use 5 epochs and not 7 as it impacts the performance.

Let us now change the gradient estimator to stochastic gradient descent and see its impact.

## Changing Gradient estimation to Stochastic Gradient Descent

In [0]:
NO_EPOCHS = 5
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='SGD',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 54s 1ms/sample - loss: -0.6117 - acc: 0.6021 - val_loss: -0.7350 - val_acc: 0.6910
Epoch 2/5
48000/48000 [==============================] - 54s 1ms/sample - loss: -0.7613 - acc: 0.7226 - val_loss: -0.7681 - val_acc: 0.7209
Epoch 3/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.7855 - acc: 0.7485 - val_loss: -0.7975 - val_acc: 0.7722
Epoch 4/5
48000/48000 [==============================] - 54s 1ms/sample - loss: -0.8012 - acc: 0.7667 - val_loss: -0.8068 - val_acc: 0.7755
Epoch 5/5
48000/48000 [==============================] - 54s 1ms/sample - loss: -0.8131 - acc: 0.7811 - val_loss: -0.8168 - val_acc: 0.7973


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.48522751784324647
Test accuracy: 0.8225


Here, we see that the test accuracy dropped to 82% and loss increased to 0.48. Hence, it is better to use Adam optimizer than stochastic gradient descent. Let us change it to Adamax and see its impact.

## Changing Gradient estimation to Adamax

In [0]:
NO_EPOCHS = 5
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='Adamax',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.8271 - acc: 0.7964 - val_loss: -0.8762 - val_acc: 0.8545
Epoch 2/5
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.8869 - acc: 0.8664 - val_loss: -0.8927 - val_acc: 0.8739
Epoch 3/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9009 - acc: 0.8835 - val_loss: -0.8975 - val_acc: 0.8788
Epoch 4/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9091 - acc: 0.8926 - val_loss: -0.9064 - val_acc: 0.8882
Epoch 5/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9181 - acc: 0.9041 - val_loss: -0.9124 - val_acc: 0.8960


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.9144185781478882
Test accuracy: 0.8985


Here, we see that the test accuracy increased to 89% and loss decreased back to -0.91. But, adam optimizer gave accuracy of 90% with loss of -0.92. Hence, Adam optimizer gives better accuracy so it is better to use it to configure the learning process.

Let us now try to change the number of layers in the network architecture and see its impact.

## Changing Network architecture - Number of layers to 2

In [0]:
NO_EPOCHS = 5
model = Sequential()
# Add convolution 2D
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  kernel_initializer='he_normal',
#                  input_shape=(IMG_ROWS, IMG_COLS, 1)))
# model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 128s 3ms/sample - loss: -0.8347 - acc: 0.8125 - val_loss: -0.8958 - val_acc: 0.8767
Epoch 2/5
48000/48000 [==============================] - 127s 3ms/sample - loss: -0.9084 - acc: 0.8921 - val_loss: -0.9158 - val_acc: 0.9003
Epoch 3/5
48000/48000 [==============================] - 128s 3ms/sample - loss: -0.9227 - acc: 0.9091 - val_loss: -0.9215 - val_acc: 0.9069
Epoch 4/5
48000/48000 [==============================] - 127s 3ms/sample - loss: -0.9330 - acc: 0.9217 - val_loss: -0.9222 - val_acc: 0.9077
Epoch 5/5
48000/48000 [==============================] - 126s 3ms/sample - loss: -0.9406 - acc: 0.9307 - val_loss: -0.9275 - val_acc: 0.9135


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.9315527265548706
Test accuracy: 0.9176


Here, we can see that decreasing the layer to 2 in the network architecture gave an accuracy of 91% which increased compared to that of 3 layer architecture with accuracy of 90%. Decreasing the layer in architecture also improves the performance and hence we will consider a 2-layer architecture.

Let us try to reduce the layer to 1 and see its impact.

## Changing Network architecture - Number of layers to 1

In [0]:
NO_EPOCHS = 5
model = Sequential()
#Add convolution 2D
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  kernel_initializer='he_normal',
#                  input_shape=(IMG_ROWS, IMG_COLS, 1)))
# model.add(MaxPooling2D((2, 2)))

# model.add(Conv2D(64, 
#                  kernel_size=(3, 3), 
#                  activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 184s 4ms/sample - loss: -0.8351 - acc: 0.8148 - val_loss: -0.9030 - val_acc: 0.8852
Epoch 2/5
48000/48000 [==============================] - 183s 4ms/sample - loss: -0.9150 - acc: 0.8998 - val_loss: -0.9136 - val_acc: 0.8982
Epoch 3/5
48000/48000 [==============================] - 182s 4ms/sample - loss: -0.9293 - acc: 0.9182 - val_loss: -0.9136 - val_acc: 0.8963
Epoch 4/5
48000/48000 [==============================] - 182s 4ms/sample - loss: -0.9414 - acc: 0.9325 - val_loss: -0.9171 - val_acc: 0.9005
Epoch 5/5
48000/48000 [==============================] - 182s 4ms/sample - loss: -0.9496 - acc: 0.9430 - val_loss: -0.9187 - val_acc: 0.9033


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.9220789421081543
Test accuracy: 0.9078


We see that the accuracy decreases back to 90% with test loss of -0.92. Hence, we keep a 2 layer architecture for our problem which gives best accuracy.

## Changing Network initializer to uniform

In [0]:
NO_EPOCHS = 5
model = Sequential()
#Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='uniform',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.7922 - acc: 0.7562 - val_loss: -0.8614 - val_acc: 0.8322
Epoch 2/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.8729 - acc: 0.8492 - val_loss: -0.8839 - val_acc: 0.8611
Epoch 3/5
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.8914 - acc: 0.8709 - val_loss: -0.8969 - val_acc: 0.8772
Epoch 4/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9029 - acc: 0.8851 - val_loss: -0.9051 - val_acc: 0.8872
Epoch 5/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9101 - acc: 0.8935 - val_loss: -0.9097 - val_acc: 0.8920


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.9103037516593933
Test accuracy: 0.8936


While changing the kernel initializer to uniform, we see that the accuracy is less - 89% and hence let us change it to random uniform and see if it has a positive impact on the accuracy.

## Changing Network initializer to random uniform

In [0]:
NO_EPOCHS = 5
model = Sequential()
#Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='random_uniform',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))


model.compile(loss=keras.losses.cosine_proximity,
              optimizer='adam',
              metrics=['accuracy'])

train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 56s 1ms/sample - loss: -0.7935 - acc: 0.7555 - val_loss: -0.8533 - val_acc: 0.8238
Epoch 2/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.8767 - acc: 0.8538 - val_loss: -0.8901 - val_acc: 0.8682
Epoch 3/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.8951 - acc: 0.8744 - val_loss: -0.9004 - val_acc: 0.8816
Epoch 4/5
48000/48000 [==============================] - 55s 1ms/sample - loss: -0.9060 - acc: 0.8884 - val_loss: -0.8955 - val_acc: 0.8732
Epoch 5/5
48000/48000 [==============================] - 54s 1ms/sample - loss: -0.9130 - acc: 0.8965 - val_loss: -0.9112 - val_acc: 0.8943


In [0]:
#Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: -0.9117093379020691
Test accuracy: 0.8959


Here the accuracy is less which is 89% and so we keep the kernel initializer as he_uniform only for our modelling to get best accuracy of 90% and test loss of -0.92

### CONCLUSION

Acheived best accuracy of 90% and test loss of -0.92 using CNN to classify the Fashion MNIST images by tuning various hyper-parameters and trying out different activation function, cost functions and even by changing network architecture.

### CONTRIBUTION

Personal contribution: 80% External references: 20%

### CITATIONS/REFERENCES

1. https://keras.io/getting-started/sequential-model-guide/
2. https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/
3. https://medium.com/@RaghavPrabhu/understanding-of-convolutional-neural-network-cnn-deep-learning-99760835f148
4. https://www.pyimagesearch.com/2018/05/07/multi-label-classification-with-keras/
5. https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6
6. https://isaacchanghau.github.io/post/loss_functions/
7. https://ieeexplore.ieee.org/document/8285338
8. https://towardsdatascience.com/types-of-optimization-algorithms-used-in-neural-networks-and-ways-to-optimize-gradient-95ae5d39529f

### LICENSE

Copyright 2019, Ami Gandhi

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.